In [40]:
import pandas as pd
import numpy as np
import glob
from bs4 import BeautifulSoup
from datetime import date
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import os
import re

In [41]:
def inicializarWebDrive():
    options = Options()
    #options.add_argument('-headless')
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("dom.disable_open_during_load", True)
    #options.set_preference("browser.download.dir", path_downloads)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

    #options.binary = path_firefox

    #service = Service(executable_path=path_geckodriver, log_output=os.path.devnull)
    service = Service(log_output=os.path.devnull)

    wd = webdriver.Firefox(service=service, options = options)
    return wd

In [42]:
def finalizarWebDrive(wd):
  wd.quit()

In [43]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [49]:
df = pd.read_csv('CONSULTA_STJ.csv', sep=';', dtype=str, keep_default_na=False)

In [50]:
df[['DESCRIÇÃO PROCESSO', 'PROCESSO', 'RECORRENTE', 'RECORRIDO', 'AGRAVANTE', 'AGRAVADO', 'INTERES.', 'ADVOGADO', 'PROCURADOR', 'LOCALIZAÇÃO', 'TIPO', 'AUTUAÇÃO', 'NÚMERO ÚNICO', 'RELATOR(A)', 'RAMO DO DIREITO', 'ASSUNTO(S)', 'TRIBUNAL DE ORIGEM',
'NÚMEROS DE ORIGEM', 'ÚLTIMA FASE', 'FASES', 'PAUTAS']] = ''

In [45]:
len(df)

783

In [51]:
display(df[10:20])

,nr_processo,id_classe,no_classe,no_classe_sigla,no_juizo,no_tipo_justica,ni_completo,no_pessoa,DESCRIÇÃO PROCESSO,PROCESSO,RECORRENTE,RECORRIDO,AGRAVANTE,AGRAVADO,INTERES.,ADVOGADO,PROCURADOR,LOCALIZAÇÃO,TIPO,AUTUAÇÃO,NÚMERO ÚNICO,RELATOR(A),RAMO DO DIREITO,ASSUNTO(S),TRIBUNAL DE ORIGEM,NÚMEROS DE ORIGEM,ÚLTIMA FASE,FASES,PAUTAS
10,50048883020194047107,1045,Agravo de Instrumento em Recurso Especial,"['Ag', 'AREsp']",STJ - Turma (6ª ),Colegiado,88628243000110,TOME S/A INDUSTRIA DE AUTO PECAS - EM RECUPERA...,,,,,,,,,,,,,,,,,,,,,
11,50015155820234047201,1151,Recurso Especial - STJ,REsp,STJ - Turma (2ª ),Colegiado,23494616000161,GODOY & RETZLAFF COMBUSTIVEIS LTDA,,,,,,,,,,,,,,,,,,,,,
12,613207,1082,Agravo em Recurso Especial,AREsp,STJ - Turma (1ª ),Colegiado,,METILDE PASCHOINA BIANCHI CARBONERA,,,,,,,,,,,,,,,,,,,,,
13,50242547520154040000,1065,Recurso Especial,REsp,STJ - Turma (2ª ),Colegiado,,HELADE COMERCIAL MARITIMA LTDA,,,,,,,,,,,,,,,,,,,,,
14,1392335,1065,Recurso Especial,REsp,STJ - Turma (1ª ),Colegiado,,COSAN COMBUSTÍVEIS LTDA,,,,,,,,,,,,,,,,,,,,,
15,00147824120104030000,1151,Recurso Especial - STJ,REsp,STJ - Turma (2ª ),Colegiado,03785696868,JUARES RICCI,,,,,,,,,,,,,,,,,,,,,
16,1336504,1065,Recurso Especial,REsp,STJ - Turma (2ª ),Colegiado,,EDUARDO FERREIRA BAGGIO,,,,,,,,,,,,,,,,,,,,,
17,50000321220134047211,1082,Agravo em Recurso Especial,AREsp,STJ - Turma (1ª ),Colegiado,83083428000172,REUNIDAS TRANSPORTADORA RODOVIARIA DE CARGAS S A,,,,,,,,,,,,,,,,,,,,,
18,50373707120134047000,1151,Recurso Especial - STJ,REsp,STJ - Turma (1ª ),Colegiado,72495065953,GILSON FELD,,,,,,,,,,,,,,,,,,,,,
19,1470774,1065,Recurso Especial,REsp,STJ - Turma (2ª ),Colegiado,,EMBAFLEX EMBALAGENS LTDA,,,,,,,,,,,,,,,,,,,,,


In [57]:
df.iloc[783]

nr_processo                                        50048883020194047107
id_classe                                                          1045
no_classe                     Agravo de Instrumento em Recurso Especial
no_classe_sigla                                         ['Ag', 'AREsp']
no_juizo                                              STJ - Turma (6ª )
no_tipo_justica                                               Colegiado
ni_completo                                              88628243000110
no_pessoa             TOME S/A INDUSTRIA DE AUTO PECAS - EM RECUPERA...
DESCRIÇÃO PROCESSO                                                     
PROCESSO                                     AGRAVO EM RECURSO ESPECIAL
RECORRENTE                                                             
RECORRIDO                                                              
AGRAVANTE                        BRANCO, MIELE & ASSOCIADOS - ADVOCACIA
AGRAVADO                                               FAZENDA N

In [ ]:
df.at[3, 'NÚMEROS DE ORIGEM']

'00127183120104040000, 127183120104040000, 200670020011007. / 1 volume, nenhum apenso.'

In [ ]:
df.at[7, 'PAUTAS']

[['02/08/2005', '14:00', '2ª Turma'],
 ['04/10/2005', '14:00', '2ª Turma'],
 ['03/10/2006', '14:00', '2ª Turma'],
 ['17/08/2006', '14:00', '2ª Turma'],
 ['19/04/2007', '14:00', '2ª Turma'],
 ['05/05/2009', '14:00', '2ª Turma'],
 ['19/03/2009', '14:00', '2ª Turma'],
 ['19/03/2009', '14:00', '2ª Turma'],
 ['13/10/2009', '14:00', '2ª Turma'],
 ['03/09/2009', '14:00', '2ª Turma'],
 ['12/04/2005', '14:00', '2ª Turma']]

In [53]:
# Função para coletar os detalhes do processo a partir da página HTML.
def coletarDadosProcesso(df, site, index):
    
    div_descricao_processo = site.find('div', {'id': 'idDescricaoProcesso'})
    df.at[index,'DESCRIÇÃO PROCESSO'] = div_descricao_processo.text
    
    # Pega os Detalhes do processo da pagina
    div = site.find('div', {'id':'idDivDetalhes'})
    linhas = div.find_all('div', {'class':'classDivLinhaDetalhes'})
    for linha in linhas:
        detail = str.strip(linha.find_all('span', {'class':'classSpanDetalhesLabel'})[0].text.replace(':', '').replace('\n', ''))
        text = str.strip(linha.find_all('span',{'class':'classSpanDetalhesTexto'})[0].text.replace('\n', ''))
        if text != '':
            if detail == '':
                df.at[index, 'NÚMEROS DE ORIGEM'] = df.at[index, 'NÚMEROS DE ORIGEM'] + ' / ' + text      
            elif df.at[index, detail] != '':
                if isinstance(df.at[index, detail], list):
                    lista = df.at[index, detail]
                    lista.append(text)
                    df.at[index, detail] = lista
                else:
                    df.at[index, detail] = [df.at[index, detail], text]    
            else:
                df.at[index, detail] = text

    # Pega as Fases do processo da pagina
    div = site.find('div', {'id':'idDivFases'})
    linhas = div.find_all('div', {'class':'classDivFaseLinha'})
    for linha in linhas:
        data = str.strip(linha.find_all('span', {'class':'classSpanFaseData'})[0].text.replace('\n','').replace('\t',''))
        hora = str.strip(linha.find_all('span',{'class':"classSpanFaseHora"})[0].text.replace('\n','').replace('\t',''))    
        texto = str.strip(linha.find_all('span',{'class': 'classSpanFaseTexto'})[0].text.replace('\n','').replace('\t',''))

        if df.at[index, 'FASES'] != '':
            if isinstance(df.at[index, 'FASES'][0], list):
                lista = df.at[index, 'FASES']
                lista.append([data, hora, texto])
                df.at[index, 'FASES'] = lista
            else:
                df.at[index, 'FASES'] = [df.at[index, 'FASES'], [data, hora, texto]] 
        else:
            df.at[index, 'FASES'] = [data, hora, texto]

    # Pega as Pautas do processo da pagina
    try:
        div = site.find('div',{'id':'idDivLinhaSemPautas'})
    except (NoSuchElementException , AttributeError) as e:
        div = site.find('div', {'id':'idCorpoPautas'})
        linhas = div.find_all('div', {'class':'clsDivLinhaPautas'})
        for linha in linhas:
            data = str.strip(linha.find_all('span', {'class':'clsLinhaItemPautas clsLinhaPautasDataJulgamento'})[0].text.replace('\n','').replace('\t',''))
            hora = str.strip(linha.find_all('span',{'class':'clsLinhaItemPautas clsLinhaPautasHoraJulgamento'})[0].text.replace('\n','').replace('\t',''))    
            orgao = str.strip(linha.find_all('span',{'class': 'clsLinhaItemPautas clsLinhaPautasOrgaoJulgamento'})[0].text.replace('\n','').replace('\t',''))

            if df.at[index, 'PAUTAS'] != '':
                if isinstance(df.at[index, 'PAUTAS'][0], list):
                    lista = df.at[index, 'PAUTAS']
                    lista.append([data, hora, orgao])
                    df.at[index, 'PAUTAS'] = lista
                else:
                    df.at[index, 'PAUTAS'] = [df.at[index, 'PAUTAS'], [data, hora, orgao]] 
            else:
                df.at[index, 'PAUTAS'] = [data, hora, orgao]

In [54]:
# Função para adicionar uma nova linha vazia ao DataFrame com valores pré-preenchidos do índice especificado.
def add_data(df, index):
    new_data = {
        'nr_processo': df.iloc[index]['nr_processo'],
        'id_classe': df.iloc[index]['id_classe'],
        'no_classe': df.iloc[index]['no_classe'],
        'no_classe_sigla': df.iloc[index]['no_classe_sigla'],
        'no_juizo': df.iloc[index]['no_juizo'],
        'no_tipo_justica': df.iloc[index]['no_tipo_justica'],
        'ni_completo': df.iloc[index]['ni_completo'],
        'no_pessoa': df.iloc[index]['no_pessoa']
    }

    new_index = len(df)
    
    df.loc[new_index] = new_data
    df.loc[new_index] = df.loc[new_index].fillna('')
    
    return new_index

In [55]:
# Função para coletar dados sobre um processo do STJ a partir do seu site e atualizar o DataFrame.
def coletarDadosSTJ(wd, df, index):
    num_processo = df.iloc[index]['nr_processo']
    wd.get(f'https://processo.stj.jus.br/processo/pesquisa/?termo={num_processo}&aplicacao=processos.ea&tipoPesquisa=tipoPesquisaGenerica&chkordem=DESC&chkMorto=MORTO')
    
    site = BeautifulSoup(wd.page_source, 'html.parser')

    div_descricao_processo = site.find('div', {'id': 'idDescricaoProcesso'})

    if div_descricao_processo:

        coletarDadosProcesso(df, site, index)
    else:    
        re_siglas = re.compile(r'^([\S]+)')
        siglas = [re_siglas.findall(s.text)[0] for s in site.find_all('span',{'class':'clsBlocoProcessoColuna clsBlocoProcessoColuna1 classSpanProcessoUF'})]
        registros = [s.text.replace('/','').replace('-','').strip() for s in site.find_all('span',{'class':'clsBlocoProcessoColuna clsBlocoProcessoColuna2 classSpanNumeroRegistro'})]

        siglas_processo = df.at[index, 'no_classe_sigla']
        sigla_final = ''

        if "'" in siglas_processo:
            siglas_processo = re.findall(r"'(.*?)'", siglas_processo)
            for sigla in siglas_processo:
                sigla = sigla.replace("'","")
                if sigla in siglas:
                    sigla_final = sigla
                    break
        else:
            sigla_final = siglas_processo

        lista_registros = []
        for i in range(len(siglas)):
            if siglas[i] == sigla_final:
                lista_registros.append(registros[i])

        if len(lista_registros) > 1:
            for i,_ in enumerate(lista_registros):
                finalizarWebDrive(wd)
                wd = inicializarWebDrive()

                if i >= 1:
                    new_index = add_data(df, index)
                else:
                    new_index = index
                
                wd.get(f'https://processo.stj.jus.br/processo/pesquisa/?num_registro={_}&aplicacao=processos.ea')
                site = BeautifulSoup(wd.page_source, 'html.parser')
                coletarDadosProcesso(df, site, new_index)
        else:
            finalizarWebDrive(wd)
            wd = inicializarWebDrive()
            wd.get(f'https://processo.stj.jus.br/processo/pesquisa/?num_registro={lista_registros[0]}&aplicacao=processos.ea')
            site = BeautifulSoup(wd.page_source, 'html.parser')
            coletarDadosProcesso(df, site, index)  
    return wd


In [ ]:
wd = inicializarWebDrive()

for index in range(0, len(df)):
  aconteceuErro = True
  timeout = 0
  while(aconteceuErro & (timeout < 10)):
    try:
      wd = coletarDadosSTJ(wd, df, index)
      aconteceuErro = False
    except Exception as e:
      print(f'Erro no Index({index}): {e}')
      timeout += 1
    sleep(2)

  #if (index + 1) % 50 == 0:
    #df.to_excel(f'{path}/FINAL/relatorio_consulta_publica_avancada_completo_parte_{(index+1)/50}.xlsx', index=False)
finalizarWebDrive(wd)
#df.to_excel(f'{path}/FINAL/relatorio_consulta_publica_avancada_completo_parte_{(index+1)/50}.xlsx', index=False)

In [56]:
wd = inicializarWebDrive()

wd = coletarDadosSTJ(wd, df, 10)
#wd = coletarDadosSTJ(wd, df, 12)

finalizarWebDrive(wd)